# Download Batch demographics

This notebook demonstrates using the Ovation API to download patient demographics and sample metadata for all samples in a workflow batch.

In [ ]:
import csv
import dateutil.parser

import ovation.session as session
import ovation.lab.workflows as workflows
from tqdm import tqdm_notebook as tqdm

Create a session object

In [ ]:
s = session.connect_lab(input('email: '))

Retrieve the workflow by Id:

In [ ]:
workflow = s.get(s.path('workflow', int(input('Workflow ID: '))))

Iterate the samples in the workflow, producing one row in the CSV output per sample:

In [ ]:
sample = workflow.samples[1]
requisition = s.get(s.path('requisition', 2151, include_org=False))

In [ ]:
requisition.requisition.physician

In [ ]:
def physician_first_name(physician):
    if physician is None or physician.name is None:
        return ""
    
    comps = physician.name.split(' ')
    if len(comps) > 1:
        return comps[0]
    else:
        return ""
    
def physician_last_name(physician):
    if physician is None or physician.name is None:
        return ""
    
    comps = physician.name.split(' ')
    if len(comps) > 1:
        return ' '.join(comps[1:])
    else:
        return comps[0]

In [ ]:
output_name = 'workflow_{}.csv'.format(workflow.workflow.id)
with open(output_name, 'w') as csvfile:
    fieldnames = ['Sample ID', 
                  'Date Received', 
                  'Patient First Name',
                  'Patient MI',
                  'Patient Last Name',
                  'Sex',
                  'DOB',
                  'MRN/Submitter ID',
                  'Additional ID #',
                  'Collection Date',
                  'Specimen Type',
                  'Patient Diagnostic Test',
                  'Physician Name']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    
    for sample in tqdm(workflow.samples):
        requisition = s.get(s.path('requisition', sample.requisition_id, include_org=False))
        
        sex = '/'.join([k for k in sample.patient.gender if sample.patient.gender[k] == True])
        
        requested_tests = '/'.join([k for k in requisition.requisition.requested_tests if requisition.requisition.requested_tests[k] == True])
        
        physician = requisition.requisition.physician
        
        row = {'Sample ID': sample.identifier, 
                  'Date Received': dateutil.parser.parse(sample.date_received).isoformat() if sample.date_received is not None else '', 
                  'Patient First Name': sample.patient.first_name,
                  'Patient Last Name': sample.patient.last_name,
                  'Sex': sex,
                  'DOB': dateutil.parser.parse(sample.patient.date_of_birth).isoformat() if sample.patient.date_of_birth is not None else '',
                  'Collection Date': dateutil.parser.parse(requisition.requisition.sample_collection_date).isoformat() if requisition.requisition.sample_collection_date is not None else '',
                  'Specimen Type': requisition.requisition.sample_type if requisition.requisition.sample_type is not None else '',
                  'Patient Diagnostic Test': requested_tests,
                  'Physician First Name': physician_first_name(physician),
                  'Physician Last Name': physician_last_name(physician)}
        writer.writerow(row)
